<a href="https://colab.research.google.com/github/elizabethavargas/baseball/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bat-Level Baseball Data

### Download Data

In [ ]:
# imports
!pip install pybaseball
from pybaseball import statcast
import pandas as pd

In [ ]:
# download data
start_date = "2024-01-01"
end_date = "2024-12-31"
data = statcast(start_dt=start_date, end_dt=end_date)

# filter data to regular games in the 2024 MLB season
data = data[data.game_type == 'R']
data

This is a large query, it may take a moment to complete


/usr/local/lib/python3.12/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


  0%|          | 0/246 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  0%|          | 1/246 [00:05<22:31,  5.52s/it]/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  1%|          | 2/246 [00:05<10:11,  2.51s/it]/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_da

In [ ]:
# keep relevant columns
df = data[['game_date', 'game_pk', 'batter', 'pitcher', 'launch_speed',
           'launch_angle', 'age_bat', 'events', 'home_team', 'pitch_type',
           'stand', 'p_throws']].copy()

# keep rows for everything that's NOT a non-batted-ball event
non_batted_ball_events = ["strikeout", "strikeout_double_play", "walk",
    "hit_by_pitch", "catcher_interf", "truncated_pa"]

df = df[~df["events"].isin(non_batted_ball_events)]

# drop na for rows without launch data
df.dropna(subset=['launch_speed', 'launch_angle', "events"], inplace=True)
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,pitch_type,stand,p_throws
198,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,FF,R,R
254,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,SL,R,R
291,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,FF,L,R
389,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,FF,R,R
450,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,CU,L,R
...,...,...,...,...,...,...,...,...,...,...,...,...
819,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,SI,L,R
1767,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,FF,R,R
1859,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,FF,R,R
1129,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,FF,R,R


In [ ]:
# combine hand pitcher threw with and the hand the batter hit with into one column
df['pitcher_batter_handedness'] = df['p_throws'] + df['stand']

# Encode home_run as binary target
df['home_run'] = df['events'] == 'home_run'
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,pitch_type,stand,p_throws,pitcher_batter_handedness,home_run
198,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,FF,R,R,RR,False
254,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,SL,R,R,RR,False
291,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,FF,L,R,RL,False
389,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,FF,R,R,RR,False
450,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,CU,L,R,RL,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,SI,L,R,RL,False
1767,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,FF,R,R,RR,False
1859,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,FF,R,R,RR,False
1129,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,FF,R,R,RR,False


In [ ]:
df.to_csv('baseball_data.csv', index=False)

In [2]:
df = pd.read_csv('baseball_data.csv')
df

,game_date,game_pk,batter,pitcher,launch_speed,launch_angle,age_bat,events,home_team,pitch_type,stand,p_throws,pitcher_batter_handedness,home_run
0,2024-09-30,747139,518595,621242,87.6,-30,35,field_out,ATL,FF,R,R,RR,False
1,2024-09-30,747139,642201,621242,72.0,21,30,single,ATL,SL,R,R,RR,False
2,2024-09-30,747139,621566,621242,76.1,58,30,field_out,ATL,FF,L,R,RL,False
3,2024-09-30,747139,578428,572955,86.7,-25,34,field_out,ATL,FF,R,R,RR,False
4,2024-09-30,747139,596019,572955,107.2,35,31,home_run,ATL,CU,L,R,RL,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123818,2024-03-20,745444,571970,506433,76.6,19,34,single,SD,SI,L,R,RL,False
123819,2024-03-20,745444,665487,607192,59.2,-31,25,field_out,SD,FF,R,R,RR,False
123820,2024-03-20,745444,593428,607192,90.9,60,32,field_out,SD,FF,R,R,RR,False
123821,2024-03-20,745444,669257,506433,86.6,33,29,field_out,SD,FF,R,R,RR,False


### Add Altitide, Weather, Time, and Field Data

In [ ]:
import os, time, json, requests
import pandas as pd

def get_game_info(game_pk, cache_dir="game_cache"):
    """Fetch and cache weather/game info for one game_pk"""
    os.makedirs(cache_dir, exist_ok=True)
    cache_file = f"{cache_dir}/{game_pk}.json"

    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            data = json.load(f)
    else:
        url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
        resp = requests.get(url)
        if resp.status_code != 200:
            return None
        data = resp.json()
        with open(cache_file, "w") as f:
            json.dump(data, f)
        time.sleep(0.5)

    gd = data.get("gameData", {})
    gi = gd.get("game", {})
    dt = gd.get("datetime", {})
    venue = gd.get("venue", {})
    teams = gd.get("teams", {})
    weather = gd.get("weather", {})
    game_info = gd.get("gameInfo", {})

    return {
        "game_pk": game_pk,
        "sky": weather.get("condition"),
        "temp": weather.get("temp"),
        "wind": weather.get("wind", ""),
        "gametype": gi.get("type"),
        "site": venue.get("name"),
        "hometeam": teams.get("home", {}).get("name"),
        "date": dt.get("dateTime"),
        "season": gi.get("season"),
        "daynight": dt.get("dayNight"),
        "starttime": game_info.get("firstPitch") or dt.get("time"),
        "location": venue.get("location", {}).get('city'),
        "elevation": venue.get("location", {}).get('elevation'),
        "attendance": game_info.get("attendance"),
        "capacity": venue.get("fieldInfo", {}).get('capacity'),
        "turfType": venue.get("fieldInfo", {}).get('turfType'),
        "roofType": venue.get("fieldInfo", {}).get('roofType'),
        "leftLine": venue.get("fieldInfo", {}).get('leftLine'),
        "leftCenter": venue.get("fieldInfo", {}).get('leftCenter'),
        "center": venue.get("fieldInfo", {}).get('center'),
        "rightCenter": venue.get("fieldInfo", {}).get('rightCenter'),
        "rightLine": venue.get("fieldInfo", {}).get('rightLine'),
    }

# Example: loop over a few games
game_ids = df["game_pk"].unique() #[746087, 746088]
records = []
for gid in game_ids:
    rec = get_game_info(gid)
    if rec:
        records.append(rec)

# merge back into Statcast df
games_df = pd.DataFrame(records)
df = df.merge(games_df, on="game_pk", how="left")
df

In [ ]:
df.to_csv('baseball_data2.csv', index=False)

In [ ]:
# convert wind str to int
df['wind_int'] = df.wind.str.split(' ').str[0]


# add barrel column
def is_barrel(row):
    ev = row['launch_speed']
    la = row['launch_angle']

    if ev < 98:
        return False

    if ev >= 116:
        return 8 <= la <= 50  # capped range
    else:
        # expansion formula
        expansion = ev - 98
        min_angle = 26 - expansion
        max_angle = 30 + expansion
        return min_angle <= la <= max_angle

df['barrel'] = df.apply(is_barrel, axis=1)
df


,batter,stand,player_name,launch_speed,launch_angle,release_speed,age_bat,game_date,events,home_team,pitch_type,game_pk,home_run,altitude,temperature,condition,wind,wind_int,barrel
0,664983,L,"Varland, Gus",84.6,-23,95.0,27,2024-05-01,field_out,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
1,664983,L,"Varland, Gus",72.0,52,94.4,27,2024-05-01,NaN,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
2,571466,L,"Varland, Gus",73.3,-10,94.7,33,2024-05-01,field_out,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
3,677942,R,"Varland, Gus",73.5,-10,89.2,25,2024-05-01,NaN,AZ,SL,747207,False,1100,88,3.0,"9 mph, L To R",9,False
4,682998,L,"Varland, Gus",98.5,12,89.0,24,2024-05-01,single,AZ,SL,747207,False,1100,88,3.0,"9 mph, L To R",9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39575,682928,L,"Gonzales, Marco",97.6,-15,90.9,24,2024-04-01,field_out,WSH,SI,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39576,457705,R,"Gore, MacKenzie",76.7,58,97.7,38,2024-04-01,NaN,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39577,668804,R,"Gore, MacKenzie",90.5,14,96.9,29,2024-04-01,single,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39578,656582,R,"Gore, MacKenzie",80.9,67,97.2,32,2024-04-01,field_out,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False


In [ ]:
# encode stand for model
df.stand = df.stand.astype('category').cat.codes

In [ ]:
# fill na
df.condition = df.condition.fillna(0)

# drop na
df_model = df.dropna(subset=['launch_speed', 'launch_angle', 'altitude', 'barrel', 'stand', 'age_bat', 'temperature', 'condition', 'wind_int']).copy()

X = df_model[['launch_speed', 'launch_angle', 'altitude', 'barrel', 'stand', 'age_bat', 'temperature', 'condition', 'wind_int']]
y = df_model['home_run']